In [3]:
import httpx
import asyncio
from tenacity import retry, stop_after_attempt, wait_fixed

base_url = "http://localhost:8000"

@retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
async def add_document_test():
    async with httpx.AsyncClient(timeout=60.0) as client:
        document_data = {
            "content": "This is a sample document about machine learning.",
            "metadata": {"source": "ML Course"}
        }
        response = await client.post(f"{base_url}/add_document", json=document_data)
        return response.json()

async def test_chatbot():
    print("Adding Document...")
    add_doc_response = await add_document_test()
    print("Response from /add_document:", add_doc_response)

# Run the test
await test_chatbot()


Adding Document...
Response from /add_document: {'message': 'Document added successfully'}


In [17]:
async def test_chatbot():
    print("Starting Chat...")
    
    conversation_id = "test-conversation"
    
    # First, let's add some test document
    test_doc = {
        "content": """Supervised learning is a type of machine learning where the algorithm learns from labeled training data. 
        In supervised learning, each training example has an input and a corresponding correct output (label). 
        The algorithm learns to map inputs to outputs based on this training data.""",
        "metadata": {"source": "ml_basics.txt"}
    }
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        # Add the document
        await client.post(f"{base_url}/add_document", json=test_doc)
        
        # Initial message
        initial_message = {
            "query": "Hello! What can you do?",
            "conversation_id": conversation_id
        }
        chat_response = await client.post(f"{base_url}/chat", json=initial_message)
        print("Response from chatbot:", chat_response.json())
        
        # Follow-up question
        follow_up_message = {
            "query": "Can you explain supervised learning?",
            "conversation_id": conversation_id
        }
        follow_up_response = await client.post(f"{base_url}/chat", json=follow_up_message)
        print("Follow-up response:", follow_up_response.json())
        
        # Delete the conversation
        delete_response  = await client.delete(f"{base_url}/conversations/{conversation_id}")
        print("Delete Response:", delete_response.json())

# Running the test using asyncio
await test_chatbot()

Starting Chat...
Response from chatbot: {'answer': "Hello! I'm here to help with information about machine learning, specifically supervised learning. How may I assist you today?", 'conversation_id': 'test-conversation', 'sources': ['ml_basics.txt - Supervised learning is a type of machine learning where the algorithm learns from labeled training d...', 'ml_basics.txt - Supervised learning is a type of machine learning where the algorithm learns from labeled training d...', 'ml_basics.txt - Supervised learning is a type of machine learning where the algorithm learns from labeled training d...', 'ml_basics.txt - Supervised learning is a type of machine learning where the algorithm learns from labeled training d...']}
Follow-up response: {'answer': 'Certainly! In supervised learning, the algorithm learns from labeled training data. Here’s a breakdown:\n\n1. **Training Data**: This includes a set of inputs (features) and corresponding correct outputs or labels.\n\n2. **Learning Process**: